In [1]:
%matplotlib qt
import scipy.stats as st
import numpy as np
from scipy import fftpack
import scipy.ndimage.filters
import matplotlib.pyplot as plt
import skimage.io
import skimage.exposure
import skimage.util
import skimage.morphology
import scipy.ndimage
import cv2 as cv
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier


In [2]:
def remove_translation(dataset):
    r = np.zeros_like(dataset)
    for i in range(0, dataset.shape[0]):
        mean_x = np.mean(dataset[i,:][::2])
        mean_y = np.mean(dataset[i,:][1::2])
        r[i][::2] = dataset[i][::2] - mean_x
        r[i][1::2] = dataset[i][1::2] - mean_y
    return r

def remove_scale(dataset):
    r = np.zeros_like(dataset)
    for i in range(0,dataset.shape[0]):
        mean_x = np.mean(dataset[i,:][::2])
        mean_y = np.mean(dataset[i,:][1::2])
        s = np.sqrt((np.sum(np.power(dataset[i,:][::2] - mean_x,2)) + np.sum(np.power(dataset[i,:][1::2] - mean_y,2)) / (dataset.shape[1] / 2)))
        r[i][::2] = (dataset[i][::2] - mean_x) / s
        r[i][1::2] = (dataset[i][1::2] - mean_y) / s 
    return r

def remove_rotation(dataset, target):
    r = np.zeros_like(dataset)
    for i in range(0, dataset.shape[0]):
        element = dataset[i,:]
        element_x = element[::2]
        element_y = element[1::2]
        target_x = target[::2]
        target_y = target[1::2]
        angle = np.arctan(np.sum(element_x * target_y - element_y * target_x) 
                          / np.sum(element_x * target_x + element_y * target_y))
        for idx,x in enumerate(element_x):
            y = element_y[idx]
            r[i,2*idx] = np.cos(angle) * x - np.sin(angle) * y
            r[i,2*idx+1] = np.sin(angle) * x + np.cos(angle) * y
    return r

In [3]:
data_train  = np.genfromtxt("input/SIPdiatomsTrain.txt", delimiter=',')
data_test  = np.genfromtxt("input/SIPdiatomsTest.txt", delimiter=',')
labels_train = np.genfromtxt("input/SIPdiatomsTrain_classes.txt", delimiter=',')
labels_test = np.genfromtxt("input/SIPdiatomsTest_classes.txt", delimiter=',')


In [4]:

data_no_trans = remove_scale(data_train)

In [5]:
data_rotate = remove_rotation(data_no_trans, np.copy(data_no_trans[0,:]))

In [6]:
fig, axes = plt.subplots(1,4)
for i in range(0,5):
    tmp = data_train[i,:]
    x = tmp[::2]
    y = tmp[1::2]
    
    # data_no_trans = remove_translation(tmp)
    # print(np.mean(data_no_trans[::2]))
    # print(np.mean(data_no_trans[1::2]))
    # 
    axes[0].set_title("Original")
    axes[0].plot(data_train[i,:][::2], data_train[i,:][1::2], 'o')
    axes[1].plot(data_no_trans[i,:][::2], data_no_trans[i,:][1::2], 'o')
    axes[1].set_title("Translation and scale removed")
    axes[2].plot(data_rotate[i,:][::2], data_rotate[i,:][1::2], 'o')
    axes[2].set_title("Rotation removed")
    axes[2].set_xlim(axes[1].get_xlim())
    axes[2].set_ylim(axes[1].get_ylim())
    
    axes[3].set_title("Rotation refference")
    axes[3].plot(data_rotate[0,:][::2], data_rotate[0,:][1::2], 'bo')
    axes[3].set_xlim(axes[1].get_xlim())
    axes[3].set_ylim(axes[1].get_ylim())

fig.show()



In [26]:
data_train_no_trans = remove_scale(data_train)
data_train_rotate = remove_rotation(data_train_no_trans, np.copy(data_train_no_trans[0,:]))

data_test_no_trans = remove_scale(data_test)
data_test_rotate = remove_rotation(data_test_no_trans, np.copy(data_train_no_trans[0,:]))


svm = SVC()
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(data_train,labels_train)

pred = knn.predict(data_test)
acc = np.sum(pred == labels_test) / labels_test.shape[0]

print(acc)


0.03214285714285714
